<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Freat%C3%ADmetros_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

# 📈 Visualización Interactiva – Freatímetros
Este panel permite seleccionar un archivo de datos de freatímetros (CSV o XLSX), elegir un freatímetro, variable, año y estilo de gráfico para analizar la evolución temporal.


In [2]:
# === VISUALIZACIÓN FREATÍMETROS ===

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# --- Widgets y salidas ---
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)",
    description="Estilo:"
)

tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500),
    description="Tamaño:"
)

grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2,
    description="Grosor:"
)

paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly,
    description="Paleta:"
)

output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_freatimetros", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png",
    description="Formato:"
)

controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# --- Función para obtener DataFrame según origen ---
def obtener_df(origen):
    try:
        return datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
    except:
        return pd.DataFrame()

# --- Actualizar FREATÍMETROS disponibles ---
def actualizar_freatimetros(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'FREATIMETRO' not in df.columns:
        freatimetro_dropdown.options = []
        return
    lista = sorted(df['FREATIMETRO'].dropna().unique())
    freatimetro_dropdown.options = ["Todos"] + list(lista)
    freatimetro_dropdown.value = "Todos"

# --- Actualizar VARIABLES y AÑOS ---
def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluir = ['FECHA', 'FREATIMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        frea = freatimetro_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        if frea != "Todos":
            df = df[df['FREATIMETRO'] == frea]
        if anio != "Todos":
            df = df[df['FECHA'].dt.year == int(anio)]

        df = df.dropna(subset=['FECHA', variable, 'FREATIMETRO'])
        if df.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        freats = sorted(df['FREATIMETRO'].unique())
        color_map = {f: paleta[i % len(paleta)] for i, f in enumerate(freats)}

        for f in freats:
            datos_f = df[df['FREATIMETRO'] == f]
            line_args = dict(width=grosor, color=color_map[f])
            marker_args = dict(color=color_map[f])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None
            stackgroup = "one" if estilo == "Área apilada" else None

            fig.add_trace(go.Scatter(
                x=datos_f['FECHA'],
                y=datos_f[variable],
                mode=modo_graf,
                name=f,
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill,
                stackgroup=stackgroup
            ))

        fig.update_layout(
            width=ancho, height=alto,
            title=f"{variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Freatímetro",
            hovermode="x unified"
        )
        fig.show()

# --- Guardar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# --- Eventos ---
origen_dropdown.observe(actualizar_freatimetros, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')

boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Inicializar y mostrar controles ---
actualizar_freatimetros()
actualizar_variables_y_anios()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Freatímetros</h2>"))
display(origen_dropdown)
display(widgets.HBox([freatimetro_dropdown, variable_dropdown, anio_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)


Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()